# Review Sentiment

In [28]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #Score of 4 or 5
            return Sentiment.POSITIVE

# Load Data

In [29]:
import json

file_name = './books_small.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text



'Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!'

# Prep Data

In [30]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

In [31]:
X_train = [x.text for x in training]
y_train = [x.sentiment for x in training]

X_test = [x.text for x in test]
y_test = [x.sentiment for x in test]

# Bags of Words Vectorization

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# fit and transform training data
X_train_vectors = vectorizer.fit_transform(X_train)

# transform test data
X_test_vectors = vectorizer.transform(X_test)



# Classification

### Linear SVM (Support Vector Machine)

In [33]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(X_train_vectors, y_train)

print(X_test[0])
clf_svm.predict(X_test_vectors[0])

Every new Myke Cole book is better than the last, and this is no exception. If you haven't read the Shadow Ops series before start with Control Point, but go ahead and order Fortress Frontier and Breach Zone as well - you're going to want them.


array(['POSITIVE'], dtype='<U8')

### Decision Tree

### Naive Bayes

### Logistic Regression